Again, we begin by importing the necessary packages:

In [2]:
import numpy as np
import keras
import tensorflow
import gym

from Function_Library import *
from Environments import *

import sys
sys.path.insert(0, '../../keras-rl')
import rl as rl
from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy, EpsGreedyQPolicy, LinearAnnealedPolicy, GreedyQPolicy
from rl.memory import SequentialMemory
from rl.callbacks import FileLogger

import json
import copy
import os
import shutil
import datetime
import pickle

/home/evert/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [102]:
# Number of runs
num_samples = 10
# Code distance
d = 5
# Physical error rate
p_phys = 0.007
# Measurement error rate
p_measurement_error = 0.007
# Number of slices in syndrome volume
num_slices = 5

Next, we set up the agent

In [121]:
def LoadEnvAndAgent(d, p_phys, p_measurement_error):
    fixed_configs_path = os.path.join(os.getcwd(),"../trained_models/d{0}_x/fixed_config.p".format(d))
    variable_configs_path = os.path.join(os.getcwd(),"../trained_models/d{0}_x/{1}/variable_config_77.p".format(d,p_phys))
    model_weights_path = os.path.join(os.getcwd(),"../trained_models/d{0}_x/{1}/final_dqn_weights.h5f".format(d,p_phys))

    static_decoder_path = os.path.join(os.getcwd(),"referee_decoders/nn_d5_X_p5")
    static_decoder = load_model(static_decoder_path)

    fixed_configs = pickle.load( open(fixed_configs_path, "rb" ) )
    variable_configs = pickle.load( open(variable_configs_path, "rb" ) )

    all_configs = {}

    for key in fixed_configs.keys():
        all_configs[key] = fixed_configs[key]

    for key in variable_configs.keys():
        all_configs[key] = variable_configs[key]
        
    env = Surface_Code_Environment_Multi_Decoding_Cycles(d=all_configs["d"], 
    p_phys=all_configs["p_phys"], 
    p_meas=all_configs["p_meas"],  
    error_model=all_configs["error_model"], 
    use_Y=all_configs["use_Y"], 
    volume_depth=all_configs["volume_depth"],
    static_decoder=static_decoder)
    
    model = build_convolutional_nn(all_configs["c_layers"],all_configs["ff_layers"], 
                               env.observation_space.shape, env.num_actions)
    memory = SequentialMemory(limit=all_configs["buffer_size"], window_length=1)
    policy = GreedyQPolicy(masked_greedy=True)
    test_policy = GreedyQPolicy(masked_greedy=True)

    # ------------------------------------------------------------------------------------------

    dqn = DQNAgent(model=model, 
                   nb_actions=env.num_actions, 
                   memory=memory, 
                   nb_steps_warmup=all_configs["learning_starts"], 
                   target_model_update=all_configs["target_network_update_freq"], 
                   policy=policy,
                   test_policy=test_policy,
                   gamma = all_configs["gamma"],
                   enable_dueling_network=all_configs["dueling"])  


    dqn.compile(Adam(lr=all_configs["learning_rate"]))
    
    dqn.model.load_weights(model_weights_path)
    return env, dqn

In [122]:
sys.path.insert(0, '../benchmark/faulty')

import simulate_planar as sp
import planar_lattice
import perfect_matching

def DecodeWithBlossom(faulty_syndrome_slices):
   # The surface code lattice
   L =planar_lattice.PlanarLattice(d)
   # Parity lattice
   PL=planar_lattice.PlanarLattice3D(d)

   # For each time-slice
   # Create corresponding L
   for t in range(len(faulty_syndrome_slices)):

      for x in range(d):
          for y in range(d):
              q0,q1 = L.positions_Q[x+y*d]

              if( faulty_syndrome_slices[t][x,y] == 1 ):
                  L.array[q0][q1][0] *= -1
              if( faulty_syndrome_slices[t][x,y] == 2 ):
                  L.array[q0][q1][0] *= -1
                  L.array[q0][q1][1] *= -1
              if( faulty_syndrome_slices[t][x,y] == 3 ):
                  L.array[q0][q1][1] *= -1

      PL.addMeasurement(L)

    # Finally, add another layer with perfect measurements
#   L.measurePlaquettes(0)
#   L.measureStars(0)
#   PL.addMeasurement(L)

#   if showTextArray==True: L.showArrayText("errors","X")

   # Find the errors
   PL.findAnyons()

   timespace=[1,1]
   # Perfect matching on the plaquettes
   matchingX = perfect_matching.match_planar_3D(d,"plaquette",PL.anyon_positions_P,timespace)
   # Perfect matching on the starts
   matchingZ = perfect_matching.match_planar_3D(d,"star",PL.anyon_positions_S,timespace)

   # Reformat the matching
   flipsX = sp.squashMatching(d,"X",matchingX)
   flipsZ = sp.squashMatching(d,"Z",matchingZ)

   # Apply the correction...
   L.apply_flip_array("Z",flipsZ)
   L.apply_flip_array("X",flipsX)

   # ...and return the result of a logical error measurement
   return L.measure_logical()

In [123]:
def DecodeWithDeepQ(dqn, env, faulty_syndrome_slices):

    # Intialize a zero'd input volume
    input_state = np.zeros((d+1, 2*d + 1, 2*d + 1),int)

    # embed and place the faulty syndrome slices in the correct place
    for j in range(d):
        input_state[j, :, :] = env.padding_syndrome(faulty_syndrome_slices[j])

    corrections = []

    still_decoding = True
    while still_decoding:

        # Fetch the suggested correction
        action = dqn.forward(input_state)

        if action not in corrections and action != env.identity_index:
            # If the action has not yet been done, or is not the identity

            # append the suggested correction to the list of corrections
            corrections.append(action)

            # Update the input state to the agent to indicate the correction it would have made
            input_state[d, :, :] = env.padding_actions(corrections)

        else:
            # decoding should stop
            still_decoding = False

    # Now we'll have to perform these corrections, and check for a logical error
    return corrections

In [124]:
def generate_faulty_syndrome_slices():
    
    # Generate a new, clean lattice
    qubits = generateSurfaceCodeLattice(d)
    # And the hidden state
    hidden_state = np.zeros((d, d), int)

    faulty_syndrome_slices = []
    for t in range(num_slices):
        # Generate error pattern
        error = generate_IIDXZ_error(d, p_phys)
        #print(error)

        # Update hidden state
        hidden_state = obtain_new_error_configuration(hidden_state, error)

        # Generate the true syndrome
        true_syndrome = generate_surface_code_syndrome_NoFT_efficient(hidden_state, qubits)
        #print(true_syndrome)

        # Generate faulty syndrome
        faulty_syndrome = generate_faulty_syndrome(true_syndrome, p_measurement_error)
        #print(faulty_syndrome)

        faulty_syndrome_slices.append(faulty_syndrome)
    
    return faulty_syndrome_slices

In [125]:
def benchmark():
    # Set the random seed
    np.random.seed(0)
    
    success_count = 0
    for i in range(num_samples):
        faulty_syndrome_slices = generate_faulty_syndrome_slices()
        
        # At this point, we have a syndrome volume that we can attempt to decode

        # Once with Blossom
        x,z = DecodeWithBlossom(faulty_syndrome_slices)
        if x==1 and z==1:
            success_count+=1

        # And then once with DeepQ
        env, dqn = LoadEnvAndAgent(d, p_phys, p_measurement_error)
        corrections = DecodeWithDeepQ(dqn, env, faulty_syndrome_slices)
        print(corrections)

    print(success_count)

In [126]:
benchmark();

[3, 21]
[]
[]
[12]
[0]
[14]
[2]
[21, 6, 11]
[]
[9, 3]
8
